In [1]:
import sqlite3 as sql
import pandas as pd
import numpy as np
import re

seed = 101

Load the dataset from the previous notebook.

In [2]:
with sql.connect('../data/toxic.db') as conn:
    df = pd.read_sql_query('select * from toxic', conn)
df.head()

,rev_id,comment,year,logged_in,ns,sample,split,min,max,avg,y
0,2232.0,This:\n:One can make an analogy in mathematica...,2002,1,article,random,train,-1.0,1.0,0.4,0
1,4216.0,"""\n\n:Clarification for you (and Zundark's ri...",2002,1,user,random,train,0.0,2.0,0.5,0
2,8953.0,Elected or Electoral? JHK,2002,0,article,random,test,0.0,1.0,0.1,0
3,26547.0,"""This is such a fun entry. Devotchka\n\nI on...",2002,1,article,random,train,0.0,2.0,0.6,0
4,28959.0,Please relate the ozone hole to increases in c...,2002,1,article,random,test,-1.0,1.0,0.2,0


Remember to isolate the train, dev, and test sets.

In [3]:
idx_train = df['split'] == 'train'
idx_dev = df['split'] == 'dev'
idx_test = df['split'] == 'test'

Let's start things off with a pretty basic model to serve as a baseline: a CountVectorizer and a LogisticRegression. We'll start with the tokenizer we developed in the exploration step.

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import SGDClassifier

def tokenizer(text):
    return re.findall(r'[a-z0-9]+', text.lower())

vect = CountVectorizer(tokenizer=tokenizer)
clf = SGDClassifier(loss='log', max_iter=100, tol=1e-6, random_state=seed)

In [5]:
X_train = df.loc[idx_train, 'comment'].values
X_dev = df.loc[idx_dev, 'comment'].values

y_train = df.loc[idx_train, 'y'].values
y_dev = df.loc[idx_dev, 'y'].values

X_train_vect = vect.fit_transform(X_train)
X_dev_vect = vect.transform(X_dev)

In [6]:
clf.fit(X_train_vect, y_train)
y_pred = clf.predict(X_dev_vect)
np.mean(y_dev==y_pred)

0.8941110557768924

Nice, we nearly got 90% on the dev set. Is accuracy a good metric for this problem though?

In [7]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_dev, y_pred)

array([[25625,   625],
       [ 2777,  3101]], dtype=int64)

In [8]:
from sklearn.metrics import classification_report

print(classification_report(y_dev, y_pred))

              precision    recall  f1-score   support

           0       0.90      0.98      0.94     26250
           1       0.83      0.53      0.65      5878

   micro avg       0.89      0.89      0.89     32128
   macro avg       0.87      0.75      0.79     32128
weighted avg       0.89      0.89      0.88     32128



The recall is pretty poor for the positive class. Let's take a look at a situation where we incorrectly classified a comment.

In [9]:
idx_error = (y_dev != y_pred) & (y_dev == 1)
print(X_dev[idx_error][1])

Removed from article: 
:An extra-large common pork sausage was named after him.  

Nah.  


That's strange, what was the average toxicity score for this instance?

In [10]:
df['hash'] = df['comment'].map(hash)
df[df['hash'] == hash(X_dev[idx_error][1])]

,rev_id,comment,year,logged_in,ns,sample,split,min,max,avg,y,hash
445,2943468.0,Removed from article: \n:An extra-large common...,2003,1,article,random,dev,-1.0,1.0,-0.1,1,753713447025845196


This may indicate problems in the dataset (or how we aggregate the scores). At this point, I would normally revisit the EDA step to review the labeling function, but let's forge ahead for now. Since we have a pretty large imbalance and the positive class is low, let's use F1 as the evaluation metric.

In [11]:
from sklearn.metrics import f1_score

f1_score(y_dev, y_pred)

0.6457725947521866

So how do we efficiently tune our hyperparameters? We can use GridSearchCV or RandomizedSearchCV, but we have a pre-defined dev set so we need to use some tricks to override the normal behavior. This is actually pretty standard for large-scale NLP problems. Cross-validation is preferred, but often not feasible for large datasets.

```
For some datasets, a pre-defined split of the data into training- and validation fold or into several cross-validation folds already exists. Using PredefinedSplit it is possible to use these folds e.g. when searching for hyperparameters.

For example, when using a validation set, set the test_fold to 0 for all samples that are part of the validation set, and to -1 for all other samples.
```

In [12]:
from sklearn.model_selection import PredefinedSplit, GridSearchCV
from sklearn.pipeline import Pipeline

df_train_dev = df[df['split'].map(lambda x: x in {'train', 'dev'})].copy()
idx = np.where(df_train_dev['split'] == 'dev', 0, -1) # See documentation above

ps = PredefinedSplit(idx)

In [13]:
ps.get_n_splits()

1

In [14]:
pipe = Pipeline([('vect', CountVectorizer(tokenizer=tokenizer)),
                 ('clf', SGDClassifier(loss='log', max_iter=100, tol=1e-6, penalty='elasticnet'))])
param_grid = {'vect__ngram_range':[(1,1), (1,2)], 'vect__min_df':[1, 2, 5, 10, 20],
              'clf__l1_ratio':[0.0, 0.1, 0.2], 'clf__class_weight':[{0:1,1:1}, {0:1,1:2}]}
gs = GridSearchCV(pipe, param_grid, scoring='f1', n_jobs=6, cv=ps, verbose=2)
gs.fit(df_train_dev['comment'].values, df_train_dev['y'].values)
print(gs.best_params_)
print(gs.best_score_)

Fitting 1 folds for each of 60 candidates, totalling 60 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:  6.9min
[Parallel(n_jobs=6)]: Done  60 out of  60 | elapsed: 12.5min finished


{'clf__class_weight': {0: 1, 1: 2}, 'clf__l1_ratio': 0.0, 'vect__min_df': 1, 'vect__ngram_range': (1, 2)}
0.7044877222692633


In [15]:
y_pred = gs.best_estimator_.predict(X_dev)
confusion_matrix(y_dev, y_pred)

array([[25933,   317],
       [  578,  5300]], dtype=int64)

In [16]:
print(classification_report(y_dev, y_pred))

              precision    recall  f1-score   support

           0       0.98      0.99      0.98     26250
           1       0.94      0.90      0.92      5878

   micro avg       0.97      0.97      0.97     32128
   macro avg       0.96      0.94      0.95     32128
weighted avg       0.97      0.97      0.97     32128



In [17]:
np.mean(y_dev==y_pred)

0.9721426792828686

Not a bad result! Let's see if adding a TfidfTransformer can make a difference. Remember, TFIDF will reduce the impact of high-frequency words and increase the impact of rare words.

In [18]:
from sklearn.feature_extraction.text import TfidfTransformer

pipe = Pipeline([('vect', CountVectorizer(tokenizer=tokenizer)),
                 ('tfidf', TfidfTransformer()),
                 ('clf', SGDClassifier(loss='log', max_iter=100, tol=1e-6, penalty='elasticnet', n_jobs=2))])
param_grid = {'vect__ngram_range':[(1,1), (1,2)], 'vect__min_df':[1, 2, 5, 10, 20],
              'clf__l1_ratio':[0.0, 0.1, 0.2], 'clf__class_weight':[{0:1,1:1}, {0:1,1:2}]}
gs = GridSearchCV(pipe, param_grid, scoring='f1', n_jobs=4, cv=ps, verbose=2)
gs.fit(df_train_dev['comment'].values, df_train_dev['y'].values)
print(gs.best_params_)
print(gs.best_score_)

Fitting 1 folds for each of 60 candidates, totalling 60 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:  8.1min
[Parallel(n_jobs=4)]: Done  60 out of  60 | elapsed: 13.8min finished


{'clf__class_weight': {0: 1, 1: 2}, 'clf__l1_ratio': 0.0, 'vect__min_df': 20, 'vect__ngram_range': (1, 1)}
0.676572751833269


In [19]:
X_dev = df.loc[idx_dev, 'comment'].values
y_dev = df.loc[idx_dev, 'y'].values

y_pred = gs.best_estimator_.predict(X_dev)
confusion_matrix(y_dev, y_pred)

array([[25356,   894],
       [ 2333,  3545]], dtype=int64)

In [20]:
print(classification_report(y_dev, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.97      0.94     26250
           1       0.80      0.60      0.69      5878

   micro avg       0.90      0.90      0.90     32128
   macro avg       0.86      0.78      0.81     32128
weighted avg       0.89      0.90      0.89     32128



In [21]:
np.mean(y_dev==y_pred)

0.8995580179282868

Let's think about some edge cases. We saw some very long single-token messages earlier. How does our model perform on them?

In [22]:
lengths = np.array([len(comment) for comment in X_dev])
idx = lengths > 4000
idx.sum()

198

In [23]:
X_dev[idx]

array(['":In an interpreted language your source code is read in command by command by a software tool, which is called the Interpreter and executed.\n\nWhat ? Command-by-command ? No interpreter is using such crappy technique nowadays. Taw\n\n=== From Interpreted Language/Talk ===\n \n:some languages (like Perl) are compiled at runtime\n\nCould you elaborate ? Taw\n\nEvery time a perl script is run, Perl compiles it and then runs the compiled code. At least, this is the usual way of using perl, although it\'s not the only way. Zundark, 2001 Dec 15\n\nYou mean compile to syntactic tree or compile to machine code ? Taw\n\n: I believe it\'s compiled to bytecode, much like Java. As of perl 5.6 (version?), it\'s possible to produce a bytecode version of a perl program to save the compilation step. I think the bytecode can be converted back to full source as well, but I\'m not as sure about that. Indigo Perl (http://www.indigoperl.com) produces a perl compiler that produces a machine execut

In [24]:
y_dev[idx]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=int64)

In [25]:
idx = lengths > 4000
y_pred = gs.best_estimator_.predict(X_dev[idx])
np.mean(y_pred == y_dev[idx])

0.8888888888888888

In [26]:
num_tokens = np.array([len(tokenizer(comment)) for comment in X_dev])
num_tokens

array([671, 404,  70, ...,  96, 268,  34])

In [27]:
idx = num_tokens == 3
X_dev[idx]

array(['Is it Zhao?  \n\n', '\n:Are you there? ]] ',
       '\n\n== Dead link removed... ==',
       ' Haggard Discography Simplified', '\n::He always is.  |  ',
       '\n\nWikipedia is great.  ', '\n\nRemoved dispute banner. ',
       '\n:Possibley Pete Townshend?  . ',
       '\n:See Amplitude modulation )   ', '}} (Third level warning) >  ',
       '\n::Ah okay, thanks! =) — •  •  ',
       '"\n\n== Scott Keith ==\n\n \'\'e\'\' "', '  didnt mean anything',
       '\n\n== Prod ==\n\nOk. Thanks.  ', '\n:See graphe paranomon.  ',
       '\n\nboobies boobies boobies !!!!!! ',
       '\n\n:Well spotted. Removed.  ',
       ' \n\n \nwell said nicholas\n\n\n',
       '\n\n\n::Woohoo, Cheers Guys. ',
       '"\n\n==Random Indefinite Block==\n  "',
       'have\n               with them.', '"\n\nSuch as ""hoteti"".   "',
       '\n:::I agree. -  /email ', '\n:Write them! -  /email ',
       '(Browning goes on . . .) ',
       '          \n                      Love,\n                       

In [28]:
y_dev[idx]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [29]:
gs.best_estimator_.predict([''])

array([0], dtype=int64)

In [30]:
gs.best_estimator_.predict(X_dev[idx])

array([0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)